In [1]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# Šis variantas pritaikytas originaliems zive įrašams, prieš naudojant į originalių
# įrašų folderį iš aplanko su npy formato įrašais reikia perrašyti papildomus failus:
# df_transl.csv, all_beats_attr_z.csv 
#
# Skriptas zive EKG pūpsnių CNN VU klasifikatoriaus testavimui ir tikslumo įvertinimui, 
# funkcijos paimamos iš aplanko zive_cnn_fda_vu_v1.py, modelis iš model_cnn_fda_vu_v1,
# testuojami duomenys iš db_folder pacientų įrašų saugyklos, jame yra ir failas all_beats_attr.
# Skripte nėra vaizdų formavimo dalies, ji yra skripte, skirtame įrašams npy formatu.
 
# Skripte yra galimybė išvesti ekstrasistolių vietas įraše.
# Dirbant su daug įrašų reiktų užblokuoti: classification = []  # Užblokuota

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys, os
from pathlib import Path
from icecream import ic
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support

from zive_util_vu import show_confusion_matrix 
from zive_util_vu import runtime, split_SubjCode
from zive_util_vu import get_freq_unique_values, zive_read_file_1ch

from zive_util_vu import get_beat_attributes
from zive_util_vu import get_userId 
from zive_util_vu import confusion_matrix_modified, zive_read_df_rpeaks

from zive_cnn_fda_vu_v1 import predict_cnn_fda_vu_v1

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)


print(tf.__version__)

print("Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui")
print('Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio')

import warnings
warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_VU'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
all_beats_attr_fname = 'all_beats_attr_z.csv'

# Failai pūpsnių klasių formavimui
selected_beats = {'N':0, 'S':1, 'V':2}
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3, 'F':3}  

# Diskretizavimo dažnis:
fs = 200

# /////////////////////////////////////////////////////////////////

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su originaliais EKG įrašais ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_selected')

# Nuoroda į modelio aplanką
# model_dir = Path(Duomenu_aplankas, 'DNN', 'best_models', 'all_ft')
model_dir = 'model_cnn_fda_vu_v1'

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Zive duomenų aplankas: ", db_folder)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) ", rec_dir)
print("Pūpsnių atributų failas:", all_beats_attr_fname)
print("Diskretizavimo dažnis: ", fs)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print("Modelio ir scaler parametrai nuskaitomas iš aplanko: ", model_dir)

print("\n")


KeyboardInterrupt: 

In [ ]:
# PASIRUOŠIMAS

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

# Naudojamų požymių sąrašas

all_features = ['RRl', 'RRr', 'RRl/RRr',
                'signal_mean', 'signal_std', 'P_val', 'Q_val', 'R_val', 'S_val', 'T_val',
                'P_pos', 'Q_pos', 'R_pos', 'S_pos', 'T_pos', 'QRS', 'PR', 'ST', 'QT', '0', '1', '2',
                '3', '4', '5', '6', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
                '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
                '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
                '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
                '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
                '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102',
                '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114',
                '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126',
                '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138',
                '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150',
                '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
                '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174',
                '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186',
                '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198',
                '199']

# Pacientų įrašų kodų sąrašas iš Zive saugyklos:
SubjCodes = \
['1000.1630737.035', '1000.1630737.056', '1000.1630737.176', '1000.1630737.219', '1001.1630737.025', '1002.1630737.068',
'1002.1630737.071', '1002.1630737.081', '1002.1630737.108', '1002.1630737.195', '1002.1630737.282', '1003.1630737.048',
'1003.1630737.055', '1003.1630737.110', '1003.1630737.144', '1003.1630737.153', '1003.1630737.192', '1003.1630737.266',
'1004.1630737.052', '1005.1630737.028', '1005.1630737.109', '1005.1630737.143', '1005.1630737.163', '1005.1630737.180',
'1005.1630737.198', '1005.1630737.206', '1005.1630737.241', '1005.1630737.267', '1005.1630737.281', '1006.1630737.050',
'1006.1630737.066', '1006.1630737.084', '1006.1630737.099', '1006.1630737.139', '1006.1630737.171', '1006.1630737.220',
'1006.1630737.236', '1006.1630737.240', '1006.1630737.243', '1007.1630737.009', '1007.1630737.140', '1007.1630737.207',
'1008.1630737.019', '1008.1630737.089', '1008.1630737.117', '1008.1630737.181', '1008.1630737.183', '1008.1630737.201',
'1008.1630737.211', '1008.1630737.216', '1008.1630737.239', '1008.1630737.279', '1009.1630737.007', '1010.1630737.003',
'1010.1630737.018', '1010.1630737.020', '1010.1630737.041', '1010.1630737.053', '1010.1630737.105', '1010.1630737.169',
'1010.1630737.244', '1010.1630737.246', '1010.1630737.262', '1011.1630737.086', '1012.1630737.013', '1012.1630737.036',
'1012.1630737.065', '1012.1630737.069', '1012.1630737.094', '1012.1630737.128', '1012.1630737.167', '1012.1630737.212',
'1012.1630737.215', '1012.1630737.264', '1013.1630737.044', '1013.1630737.057', '1013.1630737.080', '1013.1630737.125',
'1013.1630737.189', '1014.1630737.030', '1014.1630737.051', '1014.1630737.103', '1014.1630737.127', '1014.1630737.130',
'1014.1630737.154', '1014.1630737.196', '1014.1630737.202', '1014.1630737.214', '1014.1630737.288', '1015.1630737.005',
'1015.1630737.006', '1015.1630737.008', '1015.1630737.010', '1015.1630737.024', '1015.1630737.033', '1015.1630737.073',
'1015.1630737.077', '1015.1630737.161', '1015.1630737.255', '1016.1630737.038', '1016.1630737.088', '1016.1630737.095',
'1016.1630737.098', '1016.1630737.100', '1016.1630737.106', '1016.1630737.158', '1016.1630737.205', '1016.1630737.273',
'1016.1630737.276', '1017.1630737.001', '1017.1630737.029', '1017.1630737.031', '1017.1630737.046', '1017.1630737.118',
'1017.1630737.218', '1017.1630737.256', '1018.1630737.237', '1019.1630737.123', '1019.1630737.131', '1019.1630737.145',
'1019.1630737.199', '1019.1630737.209', '1019.1630737.224', '1019.1630737.248', '1019.1630737.254', '1019.1630737.272',
'1019.1630737.278', '1020.1630737.193', '1020.1630737.260', '1021.1630737.017', '1021.1630737.102', '1021.1630737.135',
'1021.1630737.138', '1021.1630737.147', '1021.1630737.150', '1021.1630737.186', '1021.1630737.283', '1022.1630737.021',
'1022.1630737.184', '1023.1630737.034', '1023.1630737.058', '1023.1630737.060', '1023.1630737.083', '1023.1630737.148',
'1023.1630737.151', '1023.1630737.172', '1023.1630737.210', '1023.1630737.234', '1023.1630737.253', '1024.1630737.251',
'1024.1630737.261', '1025.1630737.162', '1026.1630737.289', '1027.1630737.227', '1028.1630737.049', '1028.1630737.133',
'1029.1630737.040', '1029.1630737.137', '1030.1630737.027', '1030.1630737.042', '1030.1630737.045', '1030.1630737.079',
'1030.1630737.090', '1030.1630737.120', '1030.1630737.157', '1030.1630737.175', '1030.1630737.242', '1030.1630737.257',
'1031.1630737.059', '1031.1630737.076', '1031.1630737.082', '1031.1630737.093', '1031.1630737.152', '1031.1630737.187',
'1031.1630737.225', '1031.1630737.250', '1031.1630737.265', '1031.1630737.271', '1032.1630737.002', '1032.1630737.012',
'1032.1630737.032', '1032.1630737.054', '1032.1630737.197', '1032.1630737.208', '1032.1630737.229', '1032.1630737.238',
'1032.1630737.259', '1032.1630737.285', '1033.1630737.190', '1033.1630737.191', '1033.1630737.213', '1033.1630737.258',
'1033.1630737.263', '1033.1630737.268', '1034.1630737.023', '1034.1630737.064', '1034.1630737.085', '1034.1630737.097',
'1034.1630737.101', '1034.1630737.124', '1034.1630737.204', '1034.1630737.221', '1034.1630737.252', '1034.1630737.269',
'1035.1630737.016', '1035.1630737.039', '1035.1630737.043', '1035.1630737.067', '1035.1630737.194', '1035.1630737.270',
'1035.1630737.286', '1036.1630737.070', '1036.1630737.132', '1036.1630737.188', '1037.1630737.159', '1037.1630737.185',
'1038.1630737.121', '1038.1630737.156', '1038.1630737.177', '1039.1630737.114', '1040.1630737.119', '1040.1630737.129',
'1041.1630737.164', '1041.1630737.222', '1042.1630737.037', '1042.1630737.075', '1042.1630737.091', '1042.1630737.104',
'1042.1630737.134', '1042.1630737.149', '1042.1630737.170', '1042.1630737.182', '1042.1630737.203', '1042.1630737.247',
'1043.1630737.004', '1043.1630737.014', '1043.1630737.022', '1043.1630737.047', '1043.1630737.072', '1043.1630737.168',
'1043.1630737.174', '1043.1630737.230', '1043.1630737.233', '1043.1630737.280', '1044.1630737.078', '1044.1630737.112',
'1044.1630737.122', '1044.1630737.142', '1044.1630737.155', '1044.1630737.228', '1045.1630737.015', '1045.1630737.061',
'1045.1630737.062', '1045.1630737.063', '1045.1630737.126', '1045.1630737.141', '1045.1630737.173', '1045.1630737.178',
'1045.1630737.217', '1045.1630737.245', '1046.1630737.011', '1046.1630737.107', '1046.1630737.146', '1047.1630737.074',
'1047.1630737.116', '1047.1630737.160', '1047.1630737.226', '1047.1630737.232', '1047.1630737.235', '1048.1630737.092',
'1048.1630737.096', '1048.1630737.166', '1048.1630737.200', '1048.1630737.223', '1048.1630737.231', '1048.1630737.274',
'1048.1630737.275', '1048.1630737.277', '1048.1630737.287', '1049.1630737.026', '1049.1630737.087', '1049.1630737.111',
'1049.1630737.113', '1049.1630737.115', '1049.1630737.136', '1049.1630737.165', '1049.1630737.179', '1049.1630737.249',
'1049.1630737.284']

# SubjCodes = ["1000.1631134.867", "1001.1631076.286", "1002.1630844.567", "1003.1636452.484"] #Testavimui
SubjCodes = ["1049.1630737.284"] #Testavimui

# Išvedamas pacientų įrašų sąrašas
print("\nAtliekama pūpsnių pacientų įrašuose klasifikacija")
print("Klasifikuojamų įrašų sąrašas:", SubjCodes)

# Kas kiek išvedamas apdorotų sekų skaičius
show_period = 100

# Klasių simbolinių vardų sąrašas ir klasių skaičius
class_names = list(selected_beats.keys()) 
n_classes = len(selected_beats)
print(class_names)

# Nuskaitome pūpsnių atributų masyvą
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr = pd.read_csv(file_path, index_col=0, dtype = {'userNr': int, 'file_name': str,
                                                             'sample': int, 'symbol': str, 'label': int})
all_beat_indices = all_beats_attr.index

index_start = 0
validation_set_stats = pd.DataFrame(columns=['idx', 'test_label', 'pred_label', 'SubjCode'])

start_time = time.time()
# Ciklas per pacientų įrašus
for SubjCode in SubjCodes:
    userNr, file_name = split_SubjCode(SubjCode)
    # userId, recordingId =  get_rec_Id(rec_dir, userNr, recordingNr)
    print(f"\n{SubjCode} {str(file_name):>26}")

    # Paciento EKG sekų formavimas ir klasifikacija  
    file_path = Path(rec_dir, file_name) # Dirbame su originaliais duomenimis
    sign_raw = zive_read_file_1ch(file_path)  
    signal_length = sign_raw.shape[0]
    print('signal_length= ', signal_length, sign_raw.shape)
    signal = sign_raw

    # Filtruojame signalą
    # signal = signal_filter(signal=sign_raw, sampling_rate=200, lowcut=0.2, method="butterworth", order=5)

    # Nuskaitome paciento anotacijas ir jų indeksus
    df_rpeaks = zive_read_df_rpeaks(rec_dir, file_name)
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()

    # SUFORMUOJAME TESTINĮ ir PRISKIRTŲ KLASIŲ NUMERIŲ MASYVUS

    # Nereikalingiems 'U' ir 'F' suteikiam klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])

    (unique, counts) = np.unique(test_labels, return_counts=True)
    total = counts.sum()
    print("test_labels: ", unique, counts, total)

    # Surandame pradinį SubjCode įrašo indeksą faile all_beats_attr
    userNr, file_name = split_SubjCode(SubjCode)
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['file_name']==file_name)]
    # print(f"SubjCode: {SubjCode}  first elem: {selected_ind[0]} last elem: {selected_ind[-1]}  tot: {len(selected_ind)}")
    index_start = selected_ind[0]
    # print('\nSubjCode:',SubjCode, 'index_start:', index_start)   

    pred_labels = predict_cnn_fda_vu_v1(signal, atr_sample, model_dir)
    
    (unique, counts) = np.unique(pred_labels, return_counts=True)
    total = counts.sum()
    print("pred_labels: ",unique, counts, total)

    # Surandame vietas su ekstrasistolemis. 
    classification=[]
    for i, i_sample in enumerate(atr_sample):
        if ((pred_labels[i] != 0) or test_labels[i] != 0):
            classification.append({'sample':i_sample, 'annot':test_labels[i], 'pred':pred_labels[i]})

    # Vietų sąrašas išvedamas
    # Dirbant su daug įrašų sąrašo išvedimą reikia užblokuoti !!!!!!!!!!!!!!!!!!!!!!!!!!!
    # classification = []  # Užblokuota
    if (classification):
        for row in classification:
            print(f"sample: {row['sample']:>7}   annot_label: {row['annot']:>2}   pred_label: {row['pred']:>2}")  

    #  Praleisdami indeksą, jei masyvuose test_labels ir pred_labels yra reikšmė == 3,
    # suformuojame klasifikuotinų pūpsnių indeksų sąrašą
    for idx in range(len(atr_sample)):
        flag = (test_labels[idx] == 3) or (pred_labels[idx] == 3)
        if (flag == False):
            validation_set_stats = validation_set_stats.append({'idx':index_start+idx,
            'test_label':test_labels[idx],'pred_label':pred_labels[idx], 'SubjCode': SubjCode}, ignore_index=True)

    # SURANDAME TIKSLUMO PARAMETRUS
    test_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['test_label']).astype('int') 

    # print(all_beats_attr.info())

    pred_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['pred_label']).astype('int')
    confusion = confusion_matrix(test_y, pred_y)
    # print(confusion)
    prec,rec,fsc,sup = precision_recall_fscore_support(test_y, pred_y, labels=[0, 1, 2], zero_division=0)

    str1 =f"N:{int(sup[0]):>5} S:{(int(sup[1])):3} V:{int(sup[2]):3}" 
    str2 = f"  Nprec:{prec[0]:>5.2f} Nrec:{rec[0]:5.2f} Nfsc:{fsc[0]:5.2f}"
    str3 = f"  Sprec:{prec[1]:>5.2f} Srec:{rec[1]:5.2f} Sfsc:{fsc[1]:5.2f}"
    str4 = f"  Vprec:{prec[2]:>5.2f} Vrec:{rec[2]:5.2f} Vfsc:{fsc[2]:5.2f}"
    print(str1+str2+str3+str4)

    # print(len(validation_set_stats))
    # print(len(test_y))
    # print(len(pred_y))

    end_time = time.time()
    print('\n')
    runtime(end_time-start_time)

validate_ind_lst = list(validation_set_stats['idx'])
y_validate = np.array(validation_set_stats['test_label']).astype('int')
y_predicted = np.array(validation_set_stats['pred_label']).astype('int')



Atliekama pūpsnių pacientų įrašuose klasifikacija
Klasifikuojamų įrašų sąrašas: ['1000.1631134.867', '1001.1631076.286', '1002.1630844.567', '1003.1636452.484']
['N', 'S', 'V']

1000.1631134.867                1631134.867
signal_length=  127999 (127999,)
test_labels:  [0 1] [656  12] 668
pred_labels:  [0 1 2 3] [623  40   2   3] 668
sample:     205   annot_label:  0   pred_label:  3
sample:    4498   annot_label:  0   pred_label:  1
sample:    4653   annot_label:  0   pred_label:  1
sample:    7407   annot_label:  0   pred_label:  1
sample:    7641   annot_label:  0   pred_label:  1
sample:   11339   annot_label:  1   pred_label:  2
sample:   18333   annot_label:  0   pred_label:  3
sample:   18947   annot_label:  0   pred_label:  1
sample:   19100   annot_label:  0   pred_label:  1
sample:   20504   annot_label:  0   pred_label:  1
sample:   20648   annot_label:  0   pred_label:  1
sample:   21872   annot_label:  0   pred_label:  1
sample:   22000   annot_label:  0   pred_label:  1
s

In [ ]:
# MODELIO TIKSLUMO VERTINIMO IŠ VERTINIMO IMTIES REZULTATAI

print("\nMODELIO TIKSLUMO VERTINIMO iš VERTINIMO IMTIES REZULTATAI")
print("Modelis iš aplanko: ", model_dir)

cols, dist, tot = get_freq_unique_values(y_validate, ['N', 'S', 'V'])
print(f"Klasės {cols}: {dist} Suma: {tot} ")

# APIBENDRINTI REZULTATAI

print('\nAPIBENDRINTI REZULTATAI\n')

# Skaičiuojame ir išvedame klasifikavimo lentelę
confusion = confusion_matrix(y_validate, y_predicted)
pd.set_option('display.precision',3)
show_confusion_matrix(confusion, class_names)
# print('\n')

print("\nClassification Report\n")
# target_names = [key for (key, value) in selected_beats.items()]

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

print(classification_report(y_validate, y_predicted, target_names=class_names, digits=3))
report = classification_report(y_validate, y_predicted, target_names=class_names, output_dict=True)
# output_dictbool, default=False, If True, return output as dict.
df_report = pd.DataFrame(report).transpose()
# https://medium.com/@asmaiya/you-can-something-like-this-84d28e0fd31f

# Įrašome į diską
filepath = Path(model_dir, 'Apibendrinti_rezultatai.csv') 
df_report.to_csv(filepath)    
print(f'\nApibendrinti_rezultatai įrašyti į:  {filepath}')



MODELIO TIKSLUMO VERTINIMO iš VERTINIMO IMTIES REZULTATAI
Modelis iš aplanko:  model_cnn_fda_vu_v1
Klasės ['N', 'S', 'V']: [2634   22  196] Suma: 2852 

APIBENDRINTI REZULTATAI

Confusion Matrix
      N   S    V
N  2545  64   25
S     0  15    7
V     2   0  194


Normalized Confusion Matrix
      N     S     V
N 0.966 0.024 0.009
S 0.000 0.682 0.318
V 0.010 0.000 0.990

Classification Report

              precision    recall  f1-score   support

           N      0.999     0.966     0.982      2634
           S      0.190     0.682     0.297        22
           V      0.858     0.990     0.919       196

    accuracy                          0.966      2852
   macro avg      0.682     0.879     0.733      2852
weighted avg      0.983     0.966     0.973      2852


Apibendrinti_rezultatai įrašyti į:  model_cnn_fda_vu_v1/Apibendrinti_rezultatai.csv


In [ ]:
# KLAIDŲ PASISKIRSTYMAS PER PACIENTUS IR JŲ ĮRAŠUS

from zive_util_vu import zive_read_df_data, create_SubjCode
from sklearn.metrics import accuracy_score

def collect_noise_locs(rec_dir, all_beats_attr):

    noise_arr_tot = np.empty(0, dtype=int)
    
    grouped = all_beats_attr.groupby(['userNr', 'file_name'])
    for key, group in grouped:
        # print('\n',key)
        userNr = key[0]
        file_name = key[1]

        # SubjCode naudojamas atveju, kai duomenis yra formoje SubjCode.npy, SubjCode.json
        # SubjCode = create_SubjCode(userNr, file_name)
        # filepath = Path(rec_dir, SubjCode + '.json')  # Si eilute pritaikyta atvejui,
        filepath = Path(rec_dir, file_name + '.json')
        df_noises = zive_read_df_data(filepath, 'noises')

        noise_arr = np.full(shape=len(group), fill_value=0,  dtype=int)
        idx_noise = 0

        for i, row_i in group.iterrows():
            sample = row_i['sample']
            for j, row_j in df_noises.iterrows():
                if ((sample > row_j['startIndex']) & (sample < row_j['endIndex'])):
                    noise_arr[idx_noise] = 1
                    idx_noise += 1
        noise_arr_tot = np.append(noise_arr_tot, noise_arr)
    return noise_arr_tot

def get_error(y_test, y_pred):
    # Error in %
    n_errors = 0
    for idx in range(len(y_pred)):
        if (y_test[idx] != y_pred[idx]):
            n_errors += 1
    Err = float(n_errors)/len(y_pred)*100.
    Err = round(Err, 1)
    return Err

# Pasiruošimas

pd.set_option("display.max_rows", 6000, "display.max_columns", 15)
pd.set_option('display.width', 1000)

# Sukuriame ir užpildome dataframe su sekų parametrais
df_seq_errors = pd.DataFrame(columns= ['idx', 'userNr', 'file_name'])

rows_list = []
for idx in validate_ind_lst:
# Surandame  userNr, file_name, symbol
    userNr, file_name, label, symbol = get_beat_attributes(idx, all_beats_attr)
    seq_attr = {'idx': idx, 'userNr':userNr, 'file_name':file_name}
    rows_list.append(seq_attr)

# print(rows_list[:10])

# Čia įdedame informaciją, ar pūpsnys patenka į triukšmų zoną
noise_arr = collect_noise_locs(rec_dir, all_beats_attr)
noise_arr = noise_arr[validate_ind_lst]

# print(noise_arr)
# unique, counts = np.unique(noise_arr, return_counts=True)
# print(unique, counts, noise_arr.shape[0])
# print(counts.sum())

df_seq_errors = pd.DataFrame(rows_list)

df_seq_errors['labels'] = pd.Series(y_validate)
df_seq_errors['preds'] = pd.Series(y_predicted)
zeros_arr = np.zeros( y_predicted.shape[0], dtype=int)
df_seq_errors['errors'] = pd.Series(zeros_arr)  
df_seq_errors.loc[df_seq_errors['labels'] != df_seq_errors['preds'], 'errors'] = 1 
df_seq_errors['noises'] = pd.Series(noise_arr)  

# print(df_seq_errors.info())


# Rezultatų pasiskirstymas per pacientus

print("\nRezultatų pasiskirstymas per pacientus")

# Pasiruošimas
class_names = ['N', 'S', 'V']
n_classes = len(class_names)
df_user_errors = pd.DataFrame({'userNr':pd.Series(dtype='int'), 'userId':pd.Series(dtype='str'), 
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
    'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# Išgauname pacientų vidinę (eilės nr) numeraciją
grouped  = df_seq_errors.groupby(['userNr'])
userNrs = list(grouped.groups.keys())
print(f'Pacientų: {len(userNrs)}\n')

for userNr in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin

    y_test = df_seq_errors.loc[grouped.groups[userNr]]['labels'].to_numpy(dtype=int)
    y_pred = df_seq_errors.loc[grouped.groups[userNr]]['preds'].to_numpy(dtype=int)

    Err = get_error(y_test, y_pred)
    
    noise_arr = df_seq_errors.loc[grouped.groups[userNr]]['noises'].to_numpy(dtype=int)
    Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.

    # Testavimui
    # acc = accuracy_score(y_test, y_pred)
    # print(f"userNr: {userNr} recordingNr:  {recordingNr} Accuracy: {acc:.2f}")
    # cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
    # show_confusion_matrix(cnf_matrix, class_names)
    # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826

    prec,rec,fsc,sup = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1, 2], zero_division=0)
    userId = get_userId(rec_dir, userNr)

    dict_user_errors = {'userNr':int(userNr),'userId':str(userId), 
    'N':sup[0], 'S':sup[1], 'V':sup[2],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err,  'Noise%':Noise
    }
    rows_list.append(dict_user_errors)

df_user_errors =  pd.DataFrame(rows_list) 

# Išvedame suformuotą masyvą
tit1 = f"{'userNr':>6} {'userId':>24} {'N':>8} {'S':>4} {'V':>4}"
tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
print(tit1+tit2+tit3+tit4)

for idx, row in  df_user_errors.iterrows():
    str1 =f"{int(row['userNr']):>6} {str(row['userId']):>6} {int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
    str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
    str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
    str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
    print(str1+str2+str3+str4)

filepath = Path(model_dir, 'Rezultatu_pasiskirstymas_per_pacientus.csv') 
df_user_errors.to_csv(filepath)    
print(f'\nRezultatų pasiskirstymas per pacientus įrašytas į:  {filepath}')

# print(df_user_errors)




Rezultatų pasiskirstymas per pacientus
Pacientų: 4

userNr                   userId        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
  1000 613b1d673d08d4d1f3cdc8f8      653   12    0    1.00  0.95  0.98    0.25  0.83  0.38    0.00  0.00  0.00      4.8     32.6
  1001 6144c4fbbd0cc552e427535f      539    0  194    1.00  1.00  1.00    0.00  0.00  0.00    1.00  1.00  1.00      0.0      3.4
  1002 6144c5b1bd0cc5a681275363      879    0    0    1.00  0.96  0.98    0.00  0.00  0.00    0.00  0.00  0.00      3.5     67.3
  1003 6190d4b23cd1d29db2303ce9      563   10    2    1.00  0.95  0.97    0.15  0.50  0.23    0.00  0.00  0.00      6.1     78.3

Rezultatų pasiskirstymas per pacientus įrašytas į:  model_cnn_fda_vu_v1/Rezultatu_pasiskirstymas_per_pacientus.csv


In [ ]:
# Rezultatų pasiskirstymas per pacientus ir jų įrašus
# Skaičiuojama visoms 3 klasėms Precision(tikslumas), Recall (atgaminimas), Fscore (F rodiklis)
# 
# https://towardsdatascience.com/you-dont-always-have-to-loop-through-rows-in-pandas-22a970b347ac
# 
pd.set_option("display.max_rows", 6000, "display.max_columns", 18)
pd.set_option('display.width', 1000)

# Rezultatų pasiskirstymas per pacientų įrašus
print("\n\nRezultatų pasiskirstymas per pacientų įrašus")

# Pasiruošimas
class_names = ['N', 'S', 'V']
n_classes = len(class_names)
df_rec_errors = pd.DataFrame({'userNr':pd.Series(dtype='int'),
    'userId':pd.Series(dtype='str'), 'file_name':pd.Series(dtype='str'),
    'N':pd.Series(dtype='int'), 'S':pd.Series(dtype='int'), 'V':pd.Series(dtype='int'),
    'Nprec':pd.Series(dtype='float') , 'Nrec':pd.Series(dtype='float'), 'Nfsc':pd.Series(dtype='float'),
    'Sprec':pd.Series(dtype='float') , 'Srec':pd.Series(dtype='float'), 'Sfsc':pd.Series(dtype='float'),
    'Vprec':pd.Series(dtype='float') , 'Vrec':pd.Series(dtype='float'), 'Vfsc':pd.Series(dtype='float'), 
     'Err%':pd.Series(dtype='float'), 'Noise%':pd.Series(dtype='float')})
# Pandas Empty DataFrame with Specific Column Types
# https://sparkbyexamples.com/pandas/pandas-empty-dataframe-with-specific-column-types/

rows_list = []

# Sugrupuojame eilutes pagal 'userId','recordingId', suskaičiuojame, kiek kiekviename įraše
# iš viso yra klasifikuojamų sekų (labels) ir kiek padaryta klaidų (errors).
# Grupavimo objektą paverčiame į normalų dataframe objektą

grouped  = df_seq_errors.groupby(['userNr','file_name'])
print(f'Pacientų įrašų: {grouped.ngroups}')
# print(f'{grouped.size()=}')

for key in grouped.groups:
# https://stackoverflow.com/questions/62041850/looping-over-pandas-groupby-output-when-grouping-by-multiple-columns-and-missin

    # print(f'\nGroup: {key}\n{df_seq_errors.loc[grouped.groups[key]]}')
    userNr = key[0]
    file_name = key[1]
    userId = get_userId(rec_dir, userNr)

    y_test = df_seq_errors.loc[grouped.groups[key]]['labels'].to_numpy(dtype=int)
    y_pred = df_seq_errors.loc[grouped.groups[key]]['preds'].to_numpy(dtype=int)

    Err = get_error(y_test, y_pred)

    noise_arr = df_seq_errors.loc[grouped.groups[key]]['noises'].to_numpy(dtype=int)
    Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.


    # *********************** Testavimui *******************************************************************
    # acc = accuracy_score(y_test, y_pred)
    # print(f"userNr: {userNr} recordingNr:  {recordingNr} Accuracy: {acc:.2f}")
    # cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
    # show_confusion_matrix(cnf_matrix, class_names)
    # https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826
    # *********************************************************************************************************

    prec,rec,fsc,sup = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1, 2], zero_division=0)
    
    dict_rec_errors = {'userNr':int(userNr), 'userId':userId, 'file_name':file_name,
    'N':sup[0], 'S':sup[1], 'V':sup[2],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err, 'Noise%':Noise
    }
    rows_list.append(dict_rec_errors)

df_rec_errors =  pd.DataFrame(rows_list) 

# Išvedame suformuotą masyvą
grouped  = df_rec_errors.groupby('userNr')
for userNr, group in grouped:
    # print(group.dtypes)
    print("\n")
    userId = get_userId(rec_dir, userNr)
    print(f"{'userNr:'} {userNr} {'userId:'} {userId}" )
    tit1 = f"{'userNr':>6} {'file_name':>25} {'N':>8} {'S':>4} {'V':>4}"
    tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
    tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
    tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
    print(tit1+tit2+tit3+tit4)

    for idx, row in group.iterrows():
        str1 =f"{int(row['userNr']):>6} {str(row['file_name']):>26} {int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
        str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
        str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
        str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
        print(str1+str2+str3+str4)

filepath = Path(model_dir, 'Rezultatu_pasiskirstymas_per_irasus.csv') 
df_rec_errors.to_csv(filepath)    
print(f'\nRezultatų pasiskirstymas per įrašus įrašytas į:  {filepath}')




Rezultatų pasiskirstymas per pacientų įrašus
Pacientų įrašų: 4


userNr: 1000 userId: 613b1d673d08d4d1f3cdc8f8
userNr                 file_name        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
  1000                1631134.867      653   12    0    1.00  0.95  0.98    0.25  0.83  0.38    0.00  0.00  0.00      4.8     32.6


userNr: 1001 userId: 6144c4fbbd0cc552e427535f
userNr                 file_name        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
  1001                1631076.286      539    0  194    1.00  1.00  1.00    0.00  0.00  0.00    1.00  1.00  1.00      0.0      3.4


userNr: 1002 userId: 6144c5b1bd0cc5a681275363
userNr                 file_name        N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
  1002                1630844.567      879    0    0    1.00  0.96  0.98    0.00  0.00  0.00    0.00  0.00  0.00      3.5     67.3


userNr

In [ ]:
# Klasifikavimo rodiklių skaičiavimas vienam nurodytam įrašui


userNr, file_name = 1000, '1631134.867'

# userId, recordingId =  get_rec_Id(rec_dir, userNr, file_name)
print("\nZive įrašas:")
print(f"userNr: {userNr}  file_name: {file_name}")
# print(f"\nuserId: {userId}  recordingId: {recordingId}")

grouped  = df_seq_errors.groupby(['userNr','file_name'])
y_test = df_seq_errors.loc[grouped.groups[(userNr,file_name)]]['labels'].to_numpy(dtype=int)
y_pred = df_seq_errors.loc[grouped.groups[(userNr,file_name)]]['preds'].to_numpy(dtype=int)

print("\nAnotacijų pasiskirstymas įraše:")
cols, dist, tot = get_freq_unique_values(y_test, ['N', 'S', 'V'])
print(f"Klasės {cols}: {dist} Suma: {tot}\n ")

noise_arr = df_seq_errors.loc[grouped.groups[(userNr,file_name)]]['noises'].to_numpy(dtype=int)
Noise = np.sum(noise_arr, axis=0)/noise_arr.shape[0]*100.

Err = get_error(y_test, y_pred)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f} Error(%): {Err}\n ")
cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
show_confusion_matrix(cnf_matrix, class_names)
# https://towardsdatascience.com/confusion-matrix-for-your-multi-class-machine-learning-model-ff9aa3bf7826
# *********************************************************************************************************

prec,rec,fsc,sup = precision_recall_fscore_support(y_test, y_pred, labels=[0, 1, 2], zero_division=0)
row = {
    'N':sup[0], 'S':sup[1], 'V':sup[2],
    'Nprec':prec[0], 'Nrec':rec[0], 'Nfsc':fsc[0],
    'Sprec':prec[1], 'Srec':rec[1], 'Sfsc':fsc[1],
    'Vprec':prec[2], 'Vrec':rec[2], 'Vfsc':fsc[2], 
     'Err%':Err, 'Noise%':Noise
    }

print("\n")
tit1 = f"{'N':>8} {'S':>4} {'V':>4}"
tit2 = f"{'Nprec':>8} {'Nrec':>5} {'Nfsc':>5}"
tit3 = f"{'Sprec':>8} {'Srec':>5} {'Sfsc':>5}"
tit4 = f"{'Vprec':>8} {'Vrec':>5} {'Vfsc':>5} {'Err%':>8} {'Noise%':>8}"
print(tit1+tit2+tit3+tit4)

str1 =f"{int(row['N']):>8} {int(row['S']):4} {int(row['V']):4}" 
str2 = f"{row['Nprec']:>8.2f} {row['Nrec']:5.2f} {row['Nfsc']:5.2f}"
str3 = f"{row['Sprec']:>8.2f} {row['Srec']:5.2f} {row['Sfsc']:5.2f}"
str4 = f"{row['Vprec']:>8.2f} {row['Vrec']:5.2f} {row['Vfsc']:5.2f} {row['Err%']:8.1f} {row['Noise%']:8.1f}"
print(str1+str2+str3+str4)




Zive įrašas:
userNr: 1000  file_name: 1631134.867

Anotacijų pasiskirstymas įraše:
Klasės ['N', 'S', 'V']: [653  12] Suma: 665
 
Accuracy: 0.95 Error(%): 4.8
 
Confusion Matrix
     N   S  V
N  623  30  0
S    0  10  2
V    0   0  0


Zero values! Cannot calculate Normalized Confusion Matrix


       N    S    V   Nprec  Nrec  Nfsc   Sprec  Srec  Sfsc   Vprec  Vrec  Vfsc     Err%   Noise%
     653   12    0    1.00  0.95  0.98    0.25  0.83  0.38    0.00  0.00  0.00      4.8     32.6
